# Convolutional Neural Network

> ÇALIŞMA YAPARKEN DEFTER KULLANARAK TEKRAR EDELİM YOKSA HAYAL ETMEK ZOR OLUR.



### Importing the libraries

In [2]:
import tensorflow as tf # type: ignore  
from keras.src.legacy.preprocessing.image import ImageDataGenerator # type: ignore

In [3]:
tf.__version__ # Tensorflow 2 sürümünü kullanıyoruz. 

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 4000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('test_set', # Path işlemi
                                            target_size = (128, 128), # Foto boyutu aynı
                                            batch_size = 32, # aynı
                                            class_mode = 'binary')

Found 16000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN (CNN başlatılması)

In [6]:
cnn = tf.keras.models.Sequential() 

### STEP 1 - Convolution (Konvülasyon aşaması ekliyoruz.) ANN de burada gizli katmanlar eklemiştik.

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


' \n---------------------------------------------------------------- FİLTERS ------------------------------------------------------------------------------------------------------------------------------------------------\n\nFilters parametresi, konvolüsyon katmanının kaç tane filtre (Feature Detectör) kullanacağını belirtir. Her filtre, giriş görüntüsündeki özellikleri (örneğin, kenarlar, dokular) yakalayan bir öğrenilebilir parametredir.\nÖRNEK: Yukarıda 32 olarak verilmiş. Bu, katmanda 32 farklı filtre olduğunu gösterir. Her bir filtre, giriş görüntüsünden farklı özellikler yakalamaya çalışır.\n    \nDEĞİŞTİRLME DURUMU: Daha yüksek değer (filters=64, 128, vb.) daha fazla filtre demektir. Bu, daha karmaşık özelliklerin yakalanmasını sağlar, ancak modelin karmaşıklığını ve hesaplama maliyetini artırır. \nDaha düşük değer (filters=16, 8) hesaplama maliyetini düşürür ancak daha az özellik yakalanır.\n\nHESAPLAMA MAALİYETİ NEDİR ? : \nHesaplama maliyeti, filtre sayısı, filtre boyutu ve g

### STEP 2 - Pooling  - Bir havuzlama katmanını cnn e bağlıyoruz. konvolüsyon ve Pooling işlemleri ard arda bağlanmış oluyor.

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

'\nCNN\'imize MaxPooling uygulamak istersek yukarıdaki satırı Copy-Paste yapabiliriz. \n\n---------------------------------------------------------------------- POOL_SİZE -----------------------------------------------------------------------------------------------------------\n\nBir image\'e Feature Detectör uygulandıktan sonra elde edilen Feature Map\'e Max Pooling işlemi uygulanır ve Pooled Feature Map elde edilir bunu biliyoruz. \nFeature Map\'e uygulanmak istenen Max Pooling işlemi için belirli boyutlarda bir kare kullanılması gerekiyor.\nBir image\'e uygulanan konvülasyon işlemi için gereken Feature Detectör gibi burada da böyle bir kareye ihtiyaç var. \nFeature Detectör\'ün boyutlarını belirlemek için kernel_size kullanıyorduk \nburada da bu karenin boyutunu belirten parametreye pool_size denir. Bu kareye ben pool karesi diyorum.\n!!! 2 tavsiye edilen pool boyutudur !!!\n\n\n---------------------------------------------------------------------- STRİDES -------------------------

### Adding a second convolutional layer -- Tekrardan Konvolüsyon katmanı eklemek istiyoruz.

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) 
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### STEP 3 - Flattening (Düzleştirme) Bu işlemden sonra artık ANN geliştirmenin aynısını yapıyoruz.

In [10]:
cnn.add(tf.keras.layers.Flatten())

### STEP 4 - Full Connection (ANN aynısını yapabilmek için gizli katmanlar ekliyoruz.)

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### STEP 5 - Output Layer (ANN'de olduğu gibi karar verecek son Nöronu ekliyoruz.)

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN (Ağırlıklar ayarlanıyor)

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set (Modelimizi Training seti üzerinden eğitiyoruz ve aynı zamanda Test seti üzerinden değerlendiriyoruz.)

In [ ]:
history = cnn.fit(x = training_set,    
        validation_data = test_set, 
        epochs = 20)                 

## Part 4 - Eğitim sonrası analiz

In [ ]:
import matplotlib.pyplot as plt # type: ignore

# Model eğitimi sırasında history objesinde kayıt edilen accuracy ve loss değerlerini çizdiriyoruz
def plot_training_history(history):
    plt.figure(figsize=(12, 5))  # Grafik boyutunu belirliyoruz

    # Accuracy (doğruluk) grafiği
    plt.subplot(1, 2, 1)  # İlk grafiği sol tarafa yerleştiriyoruz
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='b')  # Eğitim doğruluğunu mavi ile çizdiriyoruz
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='r')  # Doğrulama doğruluğunu kırmızı ile çizdiriyoruz
    plt.title('Training and Validation Accuracy')  # Grafiğin başlığını ekliyoruz
    plt.xlabel('Epoch')  # X eksenine "Epoch" yazıyoruz
    plt.ylabel('Accuracy')  # Y eksenine "Accuracy" yazıyoruz
    plt.legend()  # Sağ alt köşeye açıklama ekliyoruz

    # Loss (kayıp) grafiği
    plt.subplot(1, 2, 2)  # İkinci grafiği sağ tarafa yerleştiriyoruz
    plt.plot(history.history['loss'], label='Training Loss', color='b')  # Eğitim kaybını mavi ile çizdiriyoruz
    plt.plot(history.history['val_loss'], label='Validation Loss', color='r')  # Doğrulama kaybını kırmızı ile çizdiriyoruz
    plt.title('Training and Validation Loss')  # Grafiğin başlığını ekliyoruz
    plt.xlabel('Epoch')  # X eksenine "Epoch" yazıyoruz
    plt.ylabel('Loss')  # Y eksenine "Loss" yazıyoruz
    plt.legend()  # Sağ üst köşeye açıklama ekliyoruz

    plt.show()  # Grafikleri ekrana getiriyoruz

# Eğitimi bitirmiş modelin test veri seti üzerindeki doğruluk ve kayıp değerlerini gösteren fonksiyon
# Test seti üzerindeki model performansını değerlendirme
def evaluate_model_on_test_set(cnn, test_set):
    # Test setini değerlendirin
    loss, accuracy = cnn.evaluate(test_set, verbose=False)
    print(f"Test Loss: {loss}")
    print(f"Test Accuracy: {accuracy}")

# Test seti üzerindeki tahminleri gösterme
def predict_and_evaluate(cnn, test_set):
    # Test setindeki tahminleri alın
    y_pred = cnn.predict(test_set)
    # İlk 3 tahmini yazdırın
    print(f"First 3 predictions: {y_pred[:3]}") 


# Şimdi bu fonksiyonları eğitim sonrası kullanım senaryosuna dahil ediyoruz
plot_training_history(history)

# Test seti üzerindeki model performansını değerlendiriyoruz
evaluate_model_on_test_set(cnn, test_set)

# Modelin test seti üzerindeki tahminlerine bakıyoruz 
predict_and_evaluate(cnn, test_set)

## Part 5 - Prediction TAHMİN AŞAMASI

In [ ]:
# BİZİM TERCİH ETTİĞİMİZ YÖNTEM 
# Opencv çünkü projelerin geliştirilebilir olmasını istiyoruz. Opencv kodlarına eklemeler yaparken Görüntülerinizi istediğiniz gibi özelleştirmek için daha fazla kontrol sağlar.
import cv2
import numpy as np 
image = cv2.imread('path/to/image.jpg') # Fotonun yolu 
image = cv2.resize(image, (128, 128))  # target_size işlemi
image = image.astype('float32') / 255.0  # Normalizasyon 
image = np.expand_dims(image, axis=0)  # Boyutu 4 boyutlu yapmak (Detayı yöntem 2 de var.) 
predictions = cnn.predict(image)
predicted_class = np.argmax(predictions, axis=1)
class_labels = {0: 'Cat', 1: 'Dog'}  # Sınıf etiketlerini tanımla 
result = class_labels[predicted_class[0]]
print(f'Tahmin Edilen Sınıf: {result}')

# Sonuçları Görselleştirme (İsteğe Bağlı)
import matplotlib.pyplot as plt

plt.imshow(cv2.cvtColor(cv2.imread('path/to/image.jpg'), cv2.COLOR_BGR2RGB))
plt.title(f'Tahmin: {result}')
plt.axis('off')
plt.show()

# MODELİ KAYDET 

In [ ]:
cnn.save('modelim1.keras')  # Modeli .keras formatında kaydeder. [ eskiden .h5 formatında kaydedermiş. ]
#cnn = load_model('modelim.keras')  # Bir sonraki kullanım için kaydedilen modeli yükler.